In [1]:
import pennylane as qml
from pennylane import numpy as np
import itertools

In [2]:
class PauliTwirling:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
    
    def twirlinglayer(self, measurements_str: str, pauli_op_n_qubits: list,
                      noise_channel: "qml.QubitChannel function", channel_paras, channel_wires,
                      circuit: "function", *args, **kwargs):
        # channel_paras: it could be a float when the noise_channel is defined by qml.AmplitudeDamping
        # or a list when the noise_channel is defined by qml.QubitChannel.
        # *args, **kwargs: parameters of circuit.
        
        measurements_split = list(measurements_str)
        
        measurements = self.__save_measurements(measurements_split)
        
        circuit(*args, **kwargs)
        
        # Here’s the direct implementation of the Pauli twirling method
        # —it’s based on the mathematical formulation itself
        # and may differ slightly from practical approaches.
        # However, our main goal here is to use the code
        # to verify the mathematical principles behind Pauli twirling.
        
        self.__add_twirling_layer(pauli_op_n_qubits)
        
        noise_channel(channel_paras, channel_wires)
        
        self.__add_twirling_layer(pauli_op_n_qubits)
        
        return qml.expval(self.__create_measurement_ops(measurements))
    
    def paulitwirling(self, dev: "qml.device for paulitwirling", measurements_str: str,
                     noise_channel: "qml.QubitChannel function", channel_paras, channel_wires,
                      circuit: "function", *args, **kwargs):
        twirled_expectation_value_lst = []
        pauli_ops_n_qubits = self.__get_pauli_ops_n_qubits()
        
        for pauli_op_n_qubits in pauli_ops_n_qubits:
            e = qml.QNode(self.twirlinglayer, dev)(measurements_str, pauli_op_n_qubits,
                                            noise_channel, channel_paras, channel_wires,
                                            circuit, *args, **kwargs)
            twirled_expectation_value_lst.append(e)
            
        e = np.mean(twirled_expectation_value_lst)
        return e
    
       
    def __save_measurements(self, measurements_split):
        measurements = []
        for i, measurement_str in enumerate(measurements_split):
            if measurement_str == "I":
                continue
            elif measurement_str == "X":
                measurements.append(qml.PauliX(i))
            elif measurement_str == "Y":
                measurements.append(qml.PauliY(i))
            elif measurement_str == "Z":
                measurements.append(qml.PauliZ(i))
            else:
                raise ValueError("measurements_str can only be 'I', 'X', 'Y' or 'Z'.")
        return measurements
    
    def __create_measurement_ops(self, measurements):
        measurement_ops = measurements[0]
        for i in range(1, len(measurements)):
            measurement_ops = measurement_ops @ measurements[i]
        return measurement_ops
    
    
    def __get_pauli_ops_n_qubits(self):
        pauli_ops = ["I", "X", "Y", "Z"]
        pauli_ops_n_qubits = list(itertools.product(pauli_ops, repeat=self.n_qubits))
        return pauli_ops_n_qubits
    
    def __add_twirling_layer(self, pauli_op_n_qubits):
        for i, pauli_op in enumerate(pauli_op_n_qubits):
            if pauli_op == "I":
                qml.Identity(wires=i)
                
            elif pauli_op == "X":
                qml.PauliX(wires=i)
                    
            elif pauli_op == "Y":
                qml.PauliY(wires=i)
                    
            elif pauli_op == "Z":
                qml.PauliZ(wires=i)
                
            else:
                print("Unexpected error!")

In [ ]:
# Great! We've successfully written the code for Pauli twirling.
# But how does one know if our code is correct?
# Next, we'll verify the correctness of the code from two aspects.

In [3]:
n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)
paulitwirling = PauliTwirling(n_qubits)
measurements_str = "IZ"

In [ ]:
# First, from the results of test_cir0 and test_cir1,
# we can see that for the model where |0⟩ and |1⟩ states pass through the amplitude damping channel,
# the expectation values after Pauli twirling are symmetric—one positive and one negative.
# This aligns with the symmetry of the Pauli channel,
# meaning the Pauli twirling successfully transformed the asymmetry of the amplitude damping channel into symmetry.
# This is a great validation.

In [4]:
def test_cir0():
    pass

def test_cir1():
    qml.PauliX(wires=1)

In [5]:
noise_channel = qml.AmplitudeDamping
gamma = 0.15
wires = 1

In [6]:
e0 = paulitwirling.paulitwirling(dev, measurements_str,
                            noise_channel, gamma, wires,
                            test_cir0)
print(e0)

0.8500000000000001


In [7]:
e1 = paulitwirling.paulitwirling(dev, measurements_str,
                            noise_channel, gamma, wires,
                            test_cir1)
print(e1)

-0.8500000000000001


In [ ]:
# On the other hand,
# we'll verify it using the relevant theorem of the Pauli channel's Z-mixed state expression.
# According to the theorem, the expectation value of a quantum state
# after passing through the Pauli channel has a linear relationship with the ideal value
# —it only differs by a coefficient.

In [ ]:
def test_cir2():
    qml.RX(0.5, wires=1)

@qml.qnode(dev)
def test_cir2_ideal():
    qml.RX(0.5, wires=1)
    return qml.expval(qml.PauliZ(1))

In [8]:
e2 = paulitwirling.paulitwirling(dev, measurements_str,
                            noise_channel, gamma, wires,
                            test_cir2)
print(e2)

0.7459451776068167


In [9]:
test_cir2_ideal()

tensor(0.87758256, requires_grad=True)

In [10]:
p = e0
# Based on the result from test_cir0, this coefficient should be 0.85, so we use test_cir2 to verify this.
print(test_cir2_ideal() * p)
print(e2)

0.7459451776068168
0.7459451776068167


In [ ]:
# From the output expectation values of test_cir2 and test_cir2_ideal,
# we can see that they differ by exactly 0.85!
# Therefore, our Pauli twirling code is most likely correct!